In [ ]:
pwd

In [ ]:
!pip uninstall -y msspackages

In [ ]:
!pip install /root/msspackages/dist/msspackages-0.0.7-py3-none-any.whl

In [ ]:
from msspackages import setup_runner
setup_runner()

In [ ]:
!python3 -m pip install -r requirements.txt

In [2]:
import numpy as np
import random
from eks_ml_pipeline import cleanup
from msspackages import Pyspark_data_ingestion, get_features
from pyspark.sql.functions import col, count, rand, get_json_object

 pyspark_node_data = Pyspark_data_ingestion("202","9", -1, -1, "384gb", "Node")
err, pyspark_node_df = pyspark_node_data.read()

#get features
features_df = get_features(",feature_group_version)
features = features_df["feature_name"].to_list()
processed_features = feature_processor.cleanup(features)

model_parameters = features_df["model_parameters"].iloc[0]
time_steps = model_parameters["time_steps"]

#filter inital node df based on request features
node_df = pyspark_node_df.select("Timestamp", "InstanceId", *processed_features)
node_df = node_df.withColumn("Timestamp",(col("Timestamp")/1000).cast("timestamp"))

# Drop NA
cleaned_node_df = node_df.na.drop(subset=processed_features)

#Quality(timestamp filtered) nodes
quality_filtered_node_df = cleaned_node_df.groupBy("InstanceId").agg(count("Timestamp").alias("timestamp_count"))
# to get data that is closer to 1min apart
quality_filtered_nodes = quality_filtered_node_df.filter(col("timestamp_count") >= 2*time_steps)

#Processed Node DF                                                      
processed_node_df = cleaned_node_df.filter(col("InstanceId").isin(quality_filtered_nodes["InstanceId"]))

node_train, node_test = processed_node_df.randomSplit(weights=[0.8,0.2], seed=200)

node_train = node_train.toPandas()

node_test = node_test.toPandas()


In [3]:
node_train.head()

,Timestamp,InstanceId,node_cpu_utilization,node_memory_utilization
0,1664424030,i-04a4b2462d91b8365,0.649978,14.384109
1,1664424032,i-00f549b1977d71421,0.777477,4.267736
2,1664424032,i-0b9338508727a6fb2,0.786083,4.950936
3,1664424034,i-083a68dd2c550b41f,0.597077,8.435321
4,1664424036,i-0c5d6a0a32608913c,7.095775,21.414884


In [4]:
node_test.head()

,Timestamp,InstanceId,node_cpu_utilization,node_memory_utilization
0,1664424034,i-0d313c9be74747d64,5.491797,15.790962
1,1664424058,i-0db8295ecf678b21c,4.510470,40.447957
2,1664424074,i-0fce1b989f63bca3c,11.449553,2.275796
3,1664424080,i-0be6bf586e60ad4f3,10.035803,3.159258
4,1664424082,i-03421a696fce8926a,0.332393,0.641663


In [5]:
from eks_ml_pipeline import write_tensor, awswrangler_pandas_dataframe_to_s3

In [6]:
awswrangler_pandas_dataframe_to_s3(node_train,  "dish-5g.core.pd.g.dp.eks.logs.e", "node_hmm_ad", "v0.0.1", "raw_training_2022_9_29")

sucess


In [7]:
awswrangler_pandas_dataframe_to_s3(node_test,  "dish-5g.core.pd.g.dp.eks.logs.e", "node_hmm_ad", "v0.0.1", "raw_testing_2022_9_29")

sucess


In [8]:
node_train.count()

Timestamp                  535834
InstanceId                 535834
node_cpu_utilization       535834
node_memory_utilization    535834
dtype: int64

In [9]:
import boto3
import io
import pandas as pd

# Read the parquet file
buffer = io.BytesIO()
s3 = boto3.resource('s3')
object = s3.Object('dish-5g.core.pd.g.dp.eks.logs.e','node_hmm_ad/v0.0.1/data/pandas_df/raw_training_2022_9_29.parquet')
object.download_fileobj(buffer)
node_training_df = pd.read_parquet(buffer)


In [10]:
node_training_df.head()

,Timestamp,InstanceId,node_cpu_utilization,node_memory_utilization
0,1664424030,i-04a4b2462d91b8365,0.649978,14.384109
1,1664424032,i-00f549b1977d71421,0.777477,4.267736
2,1664424032,i-0b9338508727a6fb2,0.786083,4.950936
3,1664424034,i-083a68dd2c550b41f,0.597077,8.435321
4,1664424036,i-0c5d6a0a32608913c,7.095775,21.414884


In [11]:
node_training_df.count()

Timestamp                  535834
InstanceId                 535834
node_cpu_utilization       535834
node_memory_utilization    535834
dtype: int64

In [12]:
n_component = 9
covariance_type = "full"
time_steps = 12
weight = 2
hour_params = 24
split_ratio = 0.8

In [13]:
features = ["node_cpu_utilization","node_memory_utilization"]
scaled_features = ["scaled_node_cpu_utilization","scaled_node_memory_utilization"]

In [14]:
node_training_df['freq'] = node_training_df.groupby('InstanceId')['InstanceId'].transform('count')

In [15]:
filtered_node_df = node_training_df[node_training_df["freq"] > 12]

In [19]:
sample_size = round(hour_params*1440*weight/time_steps)
sample_size

5760

In [21]:
selected_node_list = filtered_node_df['InstanceId'].sample(sample_size).to_list()

In [22]:
len(selected_node_list)

5760

In [25]:
from sklearn.preprocessing import StandardScaler
import random

def tensor_builder(instance_id):
    
    features = ["node_cpu_utilization","node_memory_utilization"]
    scaled_features = ["scaled_node_cpu_utilization","scaled_node_memory_utilization"]

    ##pick random df, and normalize
    node_fe_df = filtered_node_df.loc[(filtered_node_df["InstanceId"] == instance_id)]
    node_fe_df = node_fe_df.sort_values(by='Timestamp').reset_index(drop=True)
    node_fe_df_len = len(node_fe_df)

    #tensor builder
    start = random.choice(range(node_fe_df_len-12))
    node_fe_df = node_fe_df[start:start+12]

    #scaler transformations
    scaler = StandardScaler()
    node_fe_df[scaled_features] = scaler.fit_transform(node_fe_df[features])

    return node_fe_df

In [26]:
%%time
import multiprocessing

num_cores = multiprocessing.cpu_count()
print(num_cores)
results = multiprocessing.Pool(num_cores).map(tensor_builder, selected_node_list)

96
CPU times: user 1.86 s, sys: 1.17 s, total: 3.03 s
Wall time: 6 s


In [27]:
len(results)

5760

In [34]:
results[647].shape

(12, 7)

In [35]:
results[647]

,Timestamp,InstanceId,node_cpu_utilization,node_memory_utilization,freq,scaled_node_cpu_utilization,scaled_node_memory_utilization
73,1664415081,i-0fce1b989f63bca3c,11.443150,2.274137,1169,-1.473668,-1.749892
74,1664415207,i-0fce1b989f63bca3c,11.448565,2.275410,1169,-0.771999,0.028353
75,1664415255,i-0fce1b989f63bca3c,11.455987,2.275119,1169,0.189730,-0.378225
76,1664415321,i-0fce1b989f63bca3c,11.471695,2.275287,1169,2.225067,-0.142837
77,1664415383,i-0fce1b989f63bca3c,11.455607,2.275146,1169,0.140429,-0.339707
78,1664415442,i-0fce1b989f63bca3c,11.444745,2.276770,1169,-1.266896,1.928573
79,1664415493,i-0fce1b989f63bca3c,11.454979,2.275388,1169,0.059058,-0.001605
80,1664415614,i-0fce1b989f63bca3c,11.456714,2.276061,1169,0.283902,0.937805
81,1664415748,i-0fce1b989f63bca3c,11.464998,2.274988,1169,1.357317,-0.560115
82,1664415802,i-0fce1b989f63bca3c,11.448114,2.276580,1169,-0.830471,1.663227


In [36]:
node_training_df = pd.concat(results)

In [37]:
node_training_df.shape

(69120, 7)

In [38]:
awswrangler_pandas_dataframe_to_s3(node_training_df,  "dish-5g.core.pd.g.dp.eks.logs.e", "node_hmm_ad", "v0.0.1", "training_2022_9_29")

sucess
